The goal is to find matching ICESat-2 ATL03 photon data matching the times and lat/lon of the published Kd values from BGC ARGO floats (10.5281/zenodo.8228242). 

The Kd CSV file was edited after download from Zenodo. Kd calculations were sorted by date. This pre-processing could jsut as easily be done in python but I already had the csv open to look at the contents. Edited version is called Dataset_Kd_Paper_2018. 

Actions:
- Load lat/lon and time for each Kd calculation
- Check if there is a matching pass for ICESat-2 within +- x hours and  x km.
- The notebook saves pickle files of GeoDataFrames (pandas), appending the matching row of the spreadsheet of latlontimes. 
- These GDFs are the output of the icesat2.atl03sp search function. The contents of each GDF can be found here: https://slideruleearth.io/web/rtd/user_guide/ICESat-2.html#photon-segments
- Also saves a new copy of Dataset_Kd_Paper_2018_dep with only the matching kd rows.
 


In [1]:
%pip install --quiet rtree


Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install --quiet hdbscan

Note: you may need to restart the kernel to use updated packages.


In [12]:
import icepyx as ipx
from datetime import datetime, timedelta
import time
import os
import re
import glob
import pandas as pd
import numpy as np
import logging
import matplotlib.pyplot as plt

from config import workspace_path, atl03_path,\
                 other_data_path ,shoreline_data ,gebco_path ,\
                 horizontal_res ,vertical_res ,subsurface_thresh ,\
                 ignore_subsurface_height_thres ,output_path 


    
from kd_utils.data_processing import load_data, \
                                    extract_file_params, \
                                    Extract_sea_photons, \
                                    filter_photon_dataset_by_hull_area
    
from kd_utils.sea_photons_analysis import process_sea_photon_binning,\
                            get_sea_surface_height
                            
                            
from kd_utils.bathy_processing import process_subsurface_photon_filtering

from kd_utils.visualization import plot_photon_height, \
                                   plot_kd_photons, \
                                   plot_filtered_seafloor_photons
                                   
from kd_utils.Kd_analysis import process_kd_calculation

from kd_utils.interpolation import geoid_correction, \
                                    refraction_correction, \
                                    interpolate_labels, \
                                    apply_interpolation



pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)



In [19]:
def buoy_bound_box(lat,lon,buffer_km):
    # define a buffer distance around the buoy to search for icesat-2 data
    lat_buff = buffer_km/111 # convert buffer distance to frac of 1 deg lat
    lon_buff = buffer_km/(111*np.cos(lat*np.pi/180)) # convert buffer distance to frac of 1 deg lon
    # define bounding box around the buoy (WSEN)
    # example: bbox = [-108.3, 39.2, -107.8, 38.8]
    # bbox = [lon-lon_buff,lat+lat_buff,lon+lon_buff,lat-lat_buff]
    # region = sliderule.toregion(bbox)
    minx = lon - lon_buff
    miny = lat - lat_buff
    maxx = lon + lon_buff
    maxy = lat + lat_buff

    # polygon vertices: Given as longitude, latitude coordinate pairs of decimal degrees with the last entry a repeat of the first.
    poly = [(minx, miny), (maxx, miny), (maxx, maxy), (minx, maxy), (minx, miny)]

    return poly

def main():
    try:
        
        # Load IS2 and extract sea photons
        IS2_atl03_mds, IS2_atl03_attrs, IS2_atl03_beams = load_data(ATL03_h5_file_path)  
        
        ## For debugging
        #IS2_atl03_beams=['gt1r']
        
        # Check beam types and ensure we have what we want (e.g., strong beams)
        target_beam_type=['gt1l', 'gt2l', 'gt3l']        
        IS2_atl03_beams = [beam for beam in IS2_atl03_beams if beam in target_beam_type]
        if not IS2_atl03_beams:
            print('No Strong Beam')
            # sys.exit()  # Terminate the program
        else:
            print(f'Strong Beams Found: {IS2_atl03_beams}')
        
            # Extract sea photon by applying the land mask
            sea_photon_dataset = Extract_sea_photons(IS2_atl03_mds, IS2_atl03_beams, shoreline_data_path)
        
        
            # Main processing function to apply binning beam-by-beam 
            # to bin the photons data by vertical_res 25 cm and horizontal resolution 500m
            binned_dataset_sea_surface = process_sea_photon_binning(sea_photon_dataset, horizontal_res=horizontal_res, vertical_res=vertical_res)
                    
        
            # filter to get subsurface photons
            sea_surface_height, sea_surface_label, filtered_seafloor_subsurface_photon_dataset = \
                process_subsurface_photon_filtering(binned_dataset_sea_surface, subsurface_thresh, GEBCO_file_path_lists)
        
            
            # Visualization of the filtered seafloor photons
            # Uncomment the following lines to visualize if needed
            # output_path = OutputPath+'/IS2_subsurface_RM_Seafloor.jpg'
            # plot_filtered_seafloor_photons(filtered_seafloor_subsurface_dataset=filtered_seafloor_subsurface_photon_dataset, 
            #                                 sea_photon_dataset=sea_photon_dataset, 
            #                                 sea_surface_height=sea_surface_height, 
            #                                 output_path=output_path)
        
            ########################
            # Todo: move the filter to the begining
            # Filter shallow photons that may have noises from wave, afterpulse, or others
            # by setting Ignore_Subsurface_Height_Thres (i.e., -6)
            # Masked out the deep water photons by applying (filtered_seafloor_subsurface_photon_dataset < Ignore_Subsurface_Height_Thres)
            # Filter out points below the seafloor
            Final_filtered_subsurface_photon_dataset = filtered_seafloor_subsurface_photon_dataset[
                filtered_seafloor_subsurface_photon_dataset['photon_height'] < Ignore_Subsurface_Height_Thres
            ]
            
            # Final_filtered_subsurface_photon_dataset = filtered_seafloor_subsurface_photon_dataset
                   
            #Filter dataframe based on beam list
            Final_filtered_subsurface_photon_dataset = Final_filtered_subsurface_photon_dataset[Final_filtered_subsurface_photon_dataset['beam_id'].isin(target_beam_type)]
            
            ###################################
            ### Todo: move the filter to the begining
            # Steps to standardize KD calculation by filtering out 
            # segments lacking sufficient subsurface photons.
            # Approach: Use the ConvexHull of photons within 
            # each rectanglar area formed by each horizontal bin (here, it is lat_bins) and 
            # based on dataframe Final_filtered_subsurface_photon_dataset along the distance of the track
        
            # Apply filtering based on hull area if desired
            Final_filtered_subsurface_photon_dataset, convex_hull_areas, convex_hulls = filter_photon_dataset_by_hull_area(
                Final_filtered_subsurface_photon_dataset, hull_area_threshold=1500
            )
            
            # Visualization
            # Uncomment the following line to visualize if needed
            
            # plot_target_beam=['gt1l']
            # plot_convex_hulls(Final_filtered_subsurface_photon_dataset, plot_target_beam, convex_hulls, convex_hull_areas)
            
            ########################################### 
            # calculate kd and save it to table
            SubsurfacePhotonDFAddedKd = process_kd_calculation(Final_filtered_subsurface_photon_dataset)
        
        
            ## Todo move this part into a function so that it will not looks too busy here
            # Extract the timestamp using regex and use it to set the output file name
            match = re.search(r"_(\d{14})_", ATL03_h5_file_path)
            timestamp = match.group(1) if match else "unknown"
            
            output_file_path = os.path.join(OutputPath, f"{timestamp}_AddedKdDataset_strongBeams_Further.csv")
            SubsurfacePhotonDFAddedKd.to_csv(output_file_path, index=False)
        
            if 'relative_AT_dist' in filtered_seafloor_subsurface_photon_dataset.columns:
                unique_photon_dataset = filtered_seafloor_subsurface_photon_dataset[['relative_AT_dist', 'lat_bins', 'photon_height']].drop_duplicates()
                unique_photon_dataset['relative_AT_dist_center'] = unique_photon_dataset\
                                        .groupby('lat_bins', observed=False)['relative_AT_dist'].transform('mean')
                def find_closest(group):
                    if not group.empty:
                        center = group['relative_AT_dist_center'].iloc[0]
                        group['dist_to_center'] = abs(group['relative_AT_dist'] - center)
                        closest_index = group['dist_to_center'].idxmin()
                        return group.loc[[closest_index]]
                    else:
                        return pd.DataFrame(columns=group.columns)
                
                # Filter out empty or all-NA entries
                unique_photon_dataset_filtered = unique_photon_dataset.dropna(how='all', axis=1)
        
                # Group by 'lat_bins' and apply the find_closest function
                closest_to_center = unique_photon_dataset_filtered\
                                        .groupby('lat_bins', observed=False)\
                                        .apply(find_closest, include_groups=True)
                                        # .reset_index()
                                        
                # Remove the index without resetting it if 'lat_bins' already exists
                closest_to_center.index = closest_to_center.index.droplevel(0)
            
                # Ensure relevant entries are excluded before concatenation
                closest_to_center = closest_to_center.dropna(axis=1, how='all')
            
                Kd_DF_MergedDistance = closest_to_center.merge(SubsurfacePhotonDFAddedKd, on='lat_bins', how='left')
                Kd_DF_MergedDistance = Kd_DF_MergedDistance.drop(columns=['relative_AT_dist_center', 'dist_to_center'])
                
                
                # plot_kd_photons(OutputPath, timestamp, plot_target_beam, filtered_seafloor_subsurface_photon_dataset, Kd_DF_MergedDistance)
        
            logging.info("SUCCESS!")
    
    except Exception as e:
            logging.error(f"An error occurred: {e}")

# if __name__ == '__main__':
    
#     # Get arguments function
#     args = get_args()    
   
#     ATL03_h5_file_path = os.path.join(args.workspace_path, args.atl03_path, args.atl03_file)
    
#     shoreline_data_path = os.path.join(args.workspace_path, args.other_data_path, args.shoreline_data)
    
#     GEBCO_full_path = os.path.join(args.workspace_path, args.other_data_path, args.gebco_path)
    
#     OutputPath = os.path.join(args.workspace_path, args.output_path)
    
#     #Get the gebco list
#     #gebco_2024_n0.0_s-90.0_w0.0_e90.0.tif
#     GEBCO_file_pattern = os.path.join(GEBCO_full_path, "gebco_*.tif")

#     #Initialize with an empty list
#     GEBCO_file_path_lists = []
#     for GEBCO_file_path_name in glob.glob(GEBCO_file_pattern):
#         #put the find one into the list
#         GEBCO_file_path_lists.append(GEBCO_file_path_name)

    
#     # Get all other necessary parameters
#     horizontal_res = args.horizontal_res
    
#     vertical_res = args.vertical_res
    
#     subsurface_thresh = args.subsurface_thresh
    
#     Ignore_Subsurface_Height_Thres = args.ignore_subsurface_height_thres
    
#     # When all paramters ready, call the main function
#     main()


In [8]:
# load time, lat and lon
df = pd.read_csv("data/Dataset_Kd_Paper_2018.csv")
# convert matlab time to datetime objects
df["dt_float"] = pd.to_datetime(df["dt_float"]-719529,unit='d',utc=True).round('s')
# remove all rows from before icesat2 launched
df = df[df["dt_float"]>datetime.fromisoformat('2018-10-01T00:00:00Z')]
df.reset_index(drop=True, inplace=True)
# df

In [9]:
df["check_sum"] = False
# these values can be adjusted to broaden/narrow the fit btwn icesat-2 and the ground truth
search_hrs = 12
search_km = 4
for jj in range(100,len(df)):
    if jj % 100 ==0:
        print('processing '+str(jj) +'/'+str(len(df)))# give a printout every 100 for my sanity
    # define a search region around the buoy 
    lat = df['lat_float'][jj]
    lon = df['lon_float'][jj]
    
    spatial_extent = buoy_bound_box(lat,lon,search_km)
    t_start = (df['dt_float'][jj]-timedelta(hours=search_hrs))
    t_end = (df['dt_float'][jj]+timedelta(hours=search_hrs))
    short_name = 'ATL03'
    date_range = [t_start,t_end]
    region = ipx.Query(short_name, spatial_extent, date_range)
    try:
        region.download_granules('data/')
        print('success')
        df["check_sum"] = True# if data exists and is downloaded - note in the argo csv 
    except:
        continue
    

# df = pd.read_pickle('glider_matches.pkl')
df=df[df["check_sum"]==True]
print(len(df))
# df.reset_index(drop=True, inplace=True)
df.to_csv('results/Dataset_Kd_Paper_2018_dep_4km12h.csv')

processing 100/5129
Total number of data order requests is  1  for  1  granules.
Data request  1  of  1  is submitting to NSIDC


Enter your Earthdata Login username:  gloverha
Enter your Earthdata password:  ········


order ID:  5000005875967
Initial status of your order request at NSIDC is:  processing
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order is: complete
Beginning download of zipped output...
Data request 5000005875967 of  1  order(s) is downloaded.
Download complete
success
Total number of data order requests is  1  for  1  granules.
Data request  1  of  1  is submitting to NSIDC
order ID:  5000005875968
Initial status of your order request at NSIDC is:  processing
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order is: complete
Beginning download of zipped output...
Data request 5000005875968 of  1  order(s) is downloaded.
Download complete
success
Total number of data order requests is  1  for  1  granules.
Data request  1  of  1  is submitting to NSID

In [20]:
h5names = glob.glob('data/processed_*.h5')


for jj in range(len(h5names)):
    # code from chao (in Main.ipynb)
    atl03_file_inloop = h5names[jj]
    ATL03_h5_file_path = os.path.join(workspace_path, atl03_file_inloop)
    # print(ATL03_h5_file_path)
    shoreline_data_path = os.path.join(workspace_path, other_data_path, shoreline_data)
    # print(shoreline_data_path)
    GEBCO_full_path = os.path.join(workspace_path, other_data_path, gebco_path)
    # print(GEBCO_full_path)
    OutputPath = os.path.join(workspace_path, output_path)
    # print(OutputPath)
    
    
    #Get the gebco list
    GEBCO_file_pattern = os.path.join(GEBCO_full_path, "gebco_*.tif")
    
    #Initialize with an empty list
    GEBCO_file_path_lists = []
    for GEBCO_file_path_name in glob.glob(GEBCO_file_pattern):
        #put the find one into the list
        GEBCO_file_path_lists.append(GEBCO_file_path_name)
    
    # print(GEBCO_file_path_lists)
    
    # Get all other necessary parameters
    horizontal_res = horizontal_res
    vertical_res = vertical_res
    subsurface_thresh = subsurface_thresh
    Ignore_Subsurface_Height_Thres = ignore_subsurface_height_thres
    
    # Configure logging
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    
    
    # When all paramters ready, call the main function
    main()
    # delete the hefty h5 file once done with processing - only store calculated kd values
    os.remove(ATL03_h5_file_path)





2024-12-03 21:36:01,671 - INFO - ./data/processed_ATL03_20221110022321_07691706_006_01.h5
2024-12-03 21:36:01,678 - INFO - ['METADATA', 'ancillary_data', 'atlas_impulse_response', 'ds_surf_type', 'ds_xyz', 'gt1l', 'gt1r', 'gt2l', 'gt2r', 'gt3l', 'gt3r', 'orbit_info', 'quality_assessment']


Strong Beams Found: ['gt1l', 'gt2l']
Processing strong beam: gt1l
Processing strong beam: gt2l
Processing binning for beam: gt1l
Processing binning for beam: gt2l
Processing subsurface filtering for beam: gt1l
Processing subsurface filtering for beam: gt2l


2024-12-03 21:36:45,615 - INFO - Calculating Kd for beam: gt1l
2024-12-03 21:36:45,616 - INFO - Calculating Kd from filtered subsurface photon dataset
2024-12-03 21:36:45,677 - INFO - Calculating Kd for beam: gt2l
2024-12-03 21:36:45,678 - INFO - Calculating Kd from filtered subsurface photon dataset
2024-12-03 21:36:45,770 - INFO - SUCCESS!
2024-12-03 21:36:45,780 - INFO - ./data/processed_ATL03_20220228153332_10441406_006_01.h5
2024-12-03 21:36:45,786 - INFO - ['METADATA', 'ancillary_data', 'atlas_impulse_response', 'ds_surf_type', 'ds_xyz', 'gt1l', 'gt1r', 'gt2l', 'gt2r', 'gt3l', 'gt3r', 'orbit_info', 'quality_assessment']
2024-12-03 21:36:46,193 - INFO - ./data/processed_ATL03_20181031050117_04970112_006_02.h5
2024-12-03 21:36:46,200 - INFO - ['METADATA', 'ancillary_data', 'atlas_impulse_response', 'ds_surf_type', 'ds_xyz', 'gt1l', 'gt1r', 'gt2l', 'gt2r', 'gt3l', 'gt3r', 'orbit_info', 'quality_assessment']


No Strong Beam


2024-12-03 21:36:46,764 - INFO - ./data/processed_ATL03_20210918222329_13331206_006_02.h5
2024-12-03 21:36:46,770 - INFO - ['METADATA', 'ancillary_data', 'atlas_impulse_response', 'ds_surf_type', 'ds_xyz', 'gt1l', 'gt1r', 'gt2l', 'gt2r', 'gt3l', 'gt3r', 'orbit_info', 'quality_assessment']


...

2024-12-03 21:51:14,683 - INFO - ./data/processed_ATL03_20210211090957_07541006_006_01.h5
2024-12-03 21:51:14,693 - INFO - ['METADATA', 'ancillary_data', 'atlas_impulse_response', 'ds_surf_type', 'ds_xyz', 'gt1l', 'gt1r', 'gt2l', 'gt2r', 'gt3l', 'gt3r', 'orbit_info', 'quality_assessment']


No Strong Beam


2024-12-03 21:51:15,355 - INFO - ./data/processed_ATL03_20190926024902_13750403_006_02.h5
2024-12-03 21:51:15,362 - INFO - ['METADATA', 'ancillary_data', 'atlas_impulse_response', 'ds_surf_type', 'ds_xyz', 'gt1l', 'gt1r', 'gt2l', 'gt2r', 'gt3l', 'gt3r', 'orbit_info', 'quality_assessment']


No Strong Beam
Strong Beams Found: ['gt2l', 'gt3l']
Processing strong beam: gt2l
Processing strong beam: gt3l
Processing binning for beam: gt2l
Processing binning for beam: gt3l
Processing subsurface filtering for beam: gt2l
Processing subsurface filtering for beam: gt3l


2024-12-03 21:51:57,337 - INFO - Calculating Kd for beam: gt2l
2024-12-03 21:51:57,338 - INFO - Calculating Kd from filtered subsurface photon dataset
2024-12-03 21:51:57,350 - INFO - Calculating Kd for beam: gt3l
2024-12-03 21:51:57,351 - INFO - Calculating Kd from filtered subsurface photon dataset
2024-12-03 21:51:57,397 - INFO - SUCCESS!
